# Lab 1: Information Retrieval

__Students:__ 

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

Other libraries are also fine to achieve the crawling.

e.g. scrapy, beautifulsoup... 

In [44]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request

In [45]:
x = urllib.request.urlopen('https://play.google.com/store/apps/category/GAME?hl=en').read().decode('utf-8')
print(x)

<!doctype html>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" lang="en"><head><script>(function(){window.latencyTrackerTimes={clientSideStartMs:Date.now()};}).call(this);</script><script>(function(){function _DumpException(b){window.console.error(b.stack)};var e=this,m=Date.now||function(){return+new Date};var u={};function aa(b,c){if(null===c)return!1;if("contains"in b&&1==c.nodeType)return b.contains(c);if("compareDocumentPosition"in b)return b==c||!!(b.compareDocumentPosition(c)&16);for(;c&&b!=c;)c=c.parentNode;return c==b};function ba(b,c){return function(a){a||(a=window.event);return c.call(b,a)}}function w(b){b=b.target||b.srcElement;!b.getAttribute&&b.parentNode&&(b=b.parentNode);return b}var z="undefined"!=typeof navigator&&/Macintosh/.test(navigator.userAgent),ca="undefined"!=typeof navigator&&!/Opera/.test(navigator.userAgent)&&/WebKit/.test(navigator.userAgent),da={A:1,INPUT:1,TEXTAREA:1,SELECT:1,BUTTON:1};function ea(){this._mouseEventsPrevented=!0

b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

A useful online regular expression check.
__[Check your regular expression first](https://regex101.com)__.

In [46]:
import re
appreg = r'href=\"(/store/apps/details.*?)\"'
appre = re.compile(appreg)
app_url_list = re.findall(appre,x)
app_url_list = list(set(app_url_list))
print(len(app_url_list))
print(app_url_list)

78
['/store/apps/details?id=at.nerbrothers.SuperJump', '/store/apps/details?id=com.pixel.art.coloring.color.number', '/store/apps/details?id=com.vectorunit.purple.googleplay', '/store/apps/details?id=com.candysoft.HAHWVIP', '/store/apps/details?id=com.custom.spillit', '/store/apps/details?id=com.outfit7.mytalkingtomfree', '/store/apps/details?id=com.jetstartgames.chess', '/store/apps/details?id=com.magmon.cartooncraft', '/store/apps/details?id=com.daerisoft.punch', '/store/apps/details?id=com.miniclip.soccerstars', '/store/apps/details?id=com.supercell.clashroyale', '/store/apps/details?id=com.playgendary.kickthebuddy', '/store/apps/details?id=com.fingersoft.hillclimb', '/store/apps/details?id=com.herocraft.game.full.minichess', '/store/apps/details?id=com.miniclip.eightballpool', '/store/apps/details?id=com.nintendo.zara', '/store/apps/details?id=com.youmusic.magictiles', '/store/apps/details?id=bubbles.superme', '/store/apps/details?id=com.linkdesks.bubblegames.bubbleshooter', '/stor

c) Access specific webpage to get description of each app and then store the description in files.

In [47]:
descriptions = []
for app in app_url_list:
    x = urllib.request.urlopen('https://play.google.com/' + app + '&hl=en').read().decode('utf-8')
    descriptions.append(x)

In [48]:
import nltk
from functools import reduce
from itertools import chain
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

path = '/home/jacke/nltk_data/corpora/stopwords/english'
with open(path) as f:
    stopwords = f.readlines()
    
stopwords = [x.strip() for x in stopwords]

stemmer = SnowballStemmer("english", ignore_stopwords=True)

[nltk_data] Downloading package stopwords to /home/jacke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jacke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
stopwords.append('br')

In [67]:
desc = r'itemprop=\"description.*?\">.*?<div jsname=\".*?\">(.*?)</div>'
name = r'itemprop=\"name\" content=\"(.*?)\"\/>'
appre = re.compile(desc)
appname = re.compile(name)
from collections import Counter
from sklearn.preprocessing import normalize

def tokenize(d): 
    tokens = [s.lower() for s in nltk.word_tokenize(d) if s.isalpha()]
    return [stemmer.stem(w) for w in tokens if not w in stopwords]

docs = [re.findall(appre, d)[0] for d in descriptions]
names = [re.findall(appname, d)[0] for d in descriptions]
#tokens = reduce(lambda lst, d: lst + [process(d)], docs, [])

### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__ or spaCy.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [ ]:
# import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

...)Compute tdidf 
eg. Using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.
#### You can also build the tfidf matrix with other library or your own algorithm

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
transvector = TfidfVectorizer(tokenizer=tokenize, stop_words=stopwords)

corpus = docs

#[
#     'This is the first document.',
#     'This is the second second document.',
#     'And this is the third one.',
#     'Is this the first document?',]
X = transvector.fit_transform(corpus)
print(X.toarray())
print(transvector.get_feature_names())

def tfidf(corpus):
    return transvector.fit_transform(corpus)

# idfs.item.toarray() to call all 
#idfs = [tfidf(corp) for corp in tokens]

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.04677261 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
['ab', 'abil', 'abl', 'absolut', 'acceler', 'acceleromet', 'accept', 'access', 'accessori', 'accid', 'accord', 'account', 'accumul', 'accur', 'ace', 'ach', 'achiev', 'acquir', 'acrobat', 'across', 'act', 'action', 'activ', 'actor', 'actual', 'ad', 'add', 'addict', 'addit', 'adjust', 'admiss', 'adopt', 'ador', 'adrenalin', 'adult', 'advanc', 'adventur', 'advert', 'advertis', 'affili', 'affraid', 'age', 'agil', 'agre', 'agreement', 'agriculturalist', 'ah', 'ai', 'aid', 'aim', 'airship', 'alan', 'alaska', 'album', 'aldrig', 'alien', 'alla', 'alli', 'all

### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."



In [81]:
def n_max(arr, n):
    indices = arr.ravel().argsort()[-n:]
    indices = (np.unravel_index(i, arr.shape) for i in indices)
    return reversed([(arr[i], i[1]) for i in indices])

In [84]:
def query(qstring, k):
    q = transvector.transform([qstring])
    
    res = cosine_similarity(q, X)
    
    top_k = n_max(res, k)

    return top_k

In [89]:
qres = query("A new kind of Mario game that you can play with one hand.<br><br>You control Mario by tapping as he constantly runs forward. You time your taps to pull off stylish jumps, midair spins, and wall jumps to gather coins and reach the goal!<br><br>Super Mario Run can be downloaded for free and after you purchase the game, you will be able to play all the modes with no additional payment required. You can try out all four modes before purchase: World Tour, Toad Rally, Remix 10, and Kingdom Builder.<br><br>■World Tour<br>Run and jump with style to rescue Princess Peach from Bowser’s clutches! <br>Travel through plains, caverns, ghost houses, airships, castles, and more.<br><br>Clear the 24 exciting courses to rescue Princess Peach from Bowser, waiting in his castle at the end. There are many ways to enjoy the courses, such as collecting the 3 different types of colored coins or by competing for the highest score against your friends. You can try courses 1-1 to 1-4 for free.<br><br>After rescuing Princess Peach, a nine-course special world, World Star, will appear.<br><br>■Remix 10<br>Some of the shortest Super Mario Run courses you&#39;ll ever play!<br>This mode is Super Mario Run in bite-sized bursts! You&#39;ll play through 10 short courses one after the other, with the courses changing each time you play. Daisy is lost somewhere in Remix 10, so try to clear as many courses as you can to find her!<br><br>■Toad Rally<br>Show off Mario’s stylish moves, compete against your friends, and ", 5)

In [90]:
for row in qres:
    print("ID:", row[1], " - ", names[row[1]], " - ", row[0], ". Desc:", docs[row[1]][0:50], "...")

ID: 15  -  Super Mario Run  -  0.7520727586313569 . Desc: A new kind of Mario game that you can play with on ...
ID: 67  -  Talking Tom Gold Run  -  0.12629502547596153 . Desc: <b>Run to chase after the robber</b> and explore t ...
ID: 54  -  Super Jim Jump - pixel 3d  -  0.12295089881699295 . Desc: The best platform game Super Jim Jump in pixel sty ...
ID: 62  -  Temple Run 2  -  0.11931200749719824 . Desc: With over a zillion downloads, Temple Run redefine ...
ID: 33  -  Bubble Shooter  -  0.08940825377112026 . Desc: Play the classic and most addictive bubble pop gam ...


In [88]:
docs[15]

'A new kind of Mario game that you can play with one hand.<br><br>You control Mario by tapping as he constantly runs forward. You time your taps to pull off stylish jumps, midair spins, and wall jumps to gather coins and reach the goal!<br><br>Super Mario Run can be downloaded for free and after you purchase the game, you will be able to play all the modes with no additional payment required. You can try out all four modes before purchase: World Tour, Toad Rally, Remix 10, and Kingdom Builder.<br><br>■World Tour<br>Run and jump with style to rescue Princess Peach from Bowser’s clutches! <br>Travel through plains, caverns, ghost houses, airships, castles, and more.<br><br>Clear the 24 exciting courses to rescue Princess Peach from Bowser, waiting in his castle at the end. There are many ways to enjoy the courses, such as collecting the 3 different types of colored coins or by competing for the highest score against your friends. You can try courses 1-1 to 1-4 for free.<br><br>After resc